In [ ]:
from fastcore.all import *
from fastai.vision.all import *
from fastai.metrics import error_rate
from PIL import Image 

In [ ]:
path = Path("NF1 Extract August/data/train(lots)")
path.ls()

In [ ]:
codes = np.loadtxt(path/'label_names.txt', dtype=str)
codes

In [ ]:
fnames = get_image_files(path/"images")
fnames

In [ ]:
(path/"masks").ls()
#open_mask(path/"masks/2_P.jpg")

In [ ]:
def label_func(fn): return path/"masks"/f"{fn.stem}_P.png"

In [ ]:
# Verify class codes
print("Class Codes:", codes)

# Check the number of output channels in your model
#model = unet_learner(dls, resnet34)
#print("Model Output Channels:", model.model[-1][-1].out_features)

# Check a few sample class indices in your data
for batch in dls.train:
    inputs, targets = batch
    print("Sample Class Indices in Targets:", targets.unique())

In [ ]:
dls = SegmentationDataLoaders.from_label_func(
    path, bs=6, fnames = fnames, label_func = label_func, codes = codes, item_tfms=Resize(256)
)

In [ ]:
dls.show_batch(max_n=4)

In [ ]:
learn = unet_learner(dls, resnet34)


In [ ]:
learn.fine_tune(60)

In [ ]:
learn.show_results(max_n=6, figsize=(7,8))

In [ ]:
interp = SegmentationInterpretation.from_learner(learn)
interp.plot_top_losses(k=1)

In [ ]:
learn.lr_find()

In [ ]:
img = Image.open(fnames[7])
#photo = "C:/Users/hsmit/Documents/UQ/4th year/Thesis/NF1 Extract August/NF100000005/69.jpg"
#img = Image.open(photo)
img = img.resize((256,256))
prediction = learn.predict(img)
prediction[0].show(figsize=(8,8))

blend = Image.composite(img, prediction[0], 0.5)

In [ ]:
img.show(figsize=(8,8))

In [ ]:
learn.save("all_model")